# __Méthode de gradient stochastique pour l'apprentissage d'un réseau de neurones sur un problème jouet de classification .__

<h1><a id='toc'></a>Sommaire</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#0">0. Données du problème d'optimisation </a></li>
        <li><a href="#I">1. Implémentation de la méthode du gradient stochastique à taux d'apprentissage constant (rappel) </a></li>
        <li><a href="#II">2. Réduction du bruit par décroissance du taux d'apprentissage</a></li> 
        <li><a href="#III">3. Méthodes des mini-lots (mini-batch method)</a></li> 
    </ul>

Importation des bibliothèques

In [ ]:
from toynn_2023 import *
# charge aussi les bibliothèques suivantes :
#    import numpy as np
#    from numpy import random as nprd
#    from matplotlib import pyplot as plt
#    from matplotlib import cm as cm
#    from copy import deepcopy as dcp

<h2><a id='0'></a>
0.  Données du problème d'optimisation </h2> 

<a href="#toc">top</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#0">0</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#I">1.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#II">2.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#III">3.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#bot">bot.</a>

### Sélection d'un problème ###

In [ ]:
pb = ToyPb(name = "sin", bounds = (-1,1))
pb.show_border()

### Création d'un ensemble de données taguées à l'aide du problème précédent. 

In [ ]:
ndata = 1000
data = nD_data(n = ndata, pb = pb)

data.show_class()
pb.show_border('k--')
plt.legend(loc=1, fontsize=15)
plt.show()

### Apprentissage. 

_pb_ et _data_ étant donnés, on souhaite fabriquer une fonction $h$ parmi un ensemble de fonctions de la forme $h(\cdot;A)$ où le paramètre $A$ évolue librement dans $\mathbb{R}^N$ qui minimise
$$
F(A):=\dfrac1{n_d}\sum_{i=0}^{n_d-1}\ell(h(x^i;A)y^i)=\,\dfrac1{n_d}\sum_{i=0}^{n_d-1}f_i(A).
$$
où $n_d=$*data.n*, $x^i=$*data.X*[i], $y^i=$*data.Y*[i] et $\ell=$*pb.loss*. 

Plus bas on fixe un type de réseau de neurones dont les paramètres $A$ sont les poids associés aux arrêtes et aux noeuds.  Ces réseaux de neurones ont deux noeud d'entrées auxquels on assignera les valeurs $x^i_0,x^i_1$ et un noeud en sortie qui produira la valeur $h((x^i_0,x^i_1;A)$.

### Création d'un type de réseau de neurones

Le type de réseau de neurones est caractérisé par le nombre de couches et le nombre de noeuds dans chaque couche ainsi que par une fonction d'activation $\chi$.<br>
Ici on choisit 5 couches avec respectivement 2, 4, 6, 4 et 1 noeuds et on prend  $\chi(t)=tanh(t)$.

Le paramètre _grid=(-1,1,41)_ sert pour les représentation graphiques de la sortie produite par un réseau de neurones pour les entrées $(x_i,y_j)$ parcourant les noeuds de la grille régulière obtenue par discrétisation du carré $[-1,1]\times[-1,1]$ avec le pas $h=1/20$.
$$
x_i=-1+ih,\qquad y_j=-1+jh\qquad\text{ avec }0\le i,j\le 40.
$$

In [ ]:
CardNodes = (2, 5, 5, 5, 1)
nn = ToyNN(card = CardNodes, chi="tanh", grid=(-1,1,41))

# Exemple de coefficients et représentation
A= nn.create_rand()
nn.show(A)

<h2><a id='I'></a>
1.  Implémentation de la méthode du gradient stochastique à taux d'apprentissage constant </h2> 

<a href="#toc">top</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#0">0</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#I">1.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#II">2.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#III">3.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#bot">bot.</a>

Un pas de la méthode de gradient stochastique consiste à choisir aléatoirement (et uniformément) $i_k\in\{0,\dots,n_d-1\}$ puis à faire :
$$
A^{k+1}\ \longleftarrow\ A^k - \alpha\nabla f_{i_k}(A^k),
$$
où le taux d'apprentissage $\alpha>0$ est fixé.

L'indice $i_k$ tiré aléatoirement à l'étape $k$ l'est indépendemment des indices précédents $i_0,i_1,\dots,i_{k-1}$.

__Initialisation__. On définit:<br>
&nbsp;&nbsp;&nbsp;&nbsp;
($*$) Un ensemble de coefficients initial sous la forme d'une coef-list _A_ construite au hasard.<br>
&nbsp;&nbsp;&nbsp;&nbsp;
($*$) Un flottant _alpha_ correspondant au taux d'apprentissage ($\alpha=0.05$ ici).<br>
&nbsp;&nbsp;&nbsp;&nbsp;
($*$) Un nombre d'époques total _Nepoch_.<br>
&nbsp;&nbsp;&nbsp;&nbsp;
($*$) Un entier _nepoch_ initialisé  à 0 qui va représenter le nombre d'époques effectuées.<br>
&nbsp;&nbsp;&nbsp;&nbsp;
($*$) Un entier _Ndata_ représentant la taille des données.<br>
&nbsp;&nbsp;&nbsp;&nbsp;
($*$) Un entier _nepochplot_ qui indique la fréquence des représentations graphiques au cours des itérations (une représentation graphique toutes les _niterplot_ époques). <br>
&nbsp;&nbsp;&nbsp;&nbsp;
($*$) Une liste vide *Total_loss* pour stocker l'évolution de l'erreur totale au cours des itérations.

In [ ]:
# Paramètres
alpha=0.005
Nepoch=300
Ndata=data.n
nepochplot=25

nplot_per_row = 4 # nombre de figure par ligne

# Initialisations
A=nn.create_rand()
nepoch=0
Erreur =[nn.total_loss(A,data,pb=pb)]
plotpos=0

__Boucle d'optimisation__. Noter que les deux boucles sur *i\_* et sur *j\_* pourraient être rassemblées en une seule boucle. Cette décomposition ne sert qu'à compter des époques. 

In [ ]:
# Boucle d'optimisation pour la methode du gradient stochastique
for i_ in range(Nepoch):
    nepoch+=1
    for j_ in range(Ndata):
        i = nprd.randint(Ndata)
        x, y = data.X[i], data.Y[i]
        dA = nn.descent(A, x, y, alpha=alpha, pb=pb)
        nn.add(A, dA, output=False)
    
    # calcul de l'erreur et représentations graphiques
    if not nepoch%nepochplot:
        error = nn.total_loss_and_prediction(A,data,pb=pb)
        Erreur.append(error)
        if not plotpos: plt.figure(figsize=(16,4))
        plotpos+=1
        plt.subplot(1,nplot_per_row,plotpos)
        data.show_class(pred=True)
        nn.show_pred(A)
        pb.show_border('k--')
        plt.title(f"epoch: {nepoch}, Tot. loss: {error:1.5e}.", fontsize=12)
        if plotpos==nplot_per_row :  
            plt.show()
            plotpos=0
    else:
        error = nn.total_loss(A,data,pb=pb)
        Erreur.append(error)

__Étude graphique de l'évolution de l'erreur totale__

In [ ]:
## Représentations graphiques de l'évolution de l'erreur au cours des époques.
nn.show(A)

print(f"Erreur initiale : {Erreur[0]:1.5e}")

plt.figure(figsize=(16,6))
plt.subplot(121)
debut = 1
plt.plot(np.linspace(debut, nepoch-1,nepoch-debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")

plt.subplot(122)
debut = nepoch//2
plt.plot(np.linspace(debut, nepoch-1,nepoch-debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")
plt.show()

plt.figure(figsize=(16,6))
plt.subplot(121)
debut = 3*(nepoch//4)
plt.plot(np.linspace(debut, nepoch-1,nepoch - debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")

plt.subplot(122)
debut = 7*(nepoch//8)
plt.plot(np.linspace(debut, nepoch-1,nepoch - debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")
plt.show()

**Exercice 1.**

(a) Commentez l'évolution de l'erreur. Donnez des explications.

(b) Changez la valeur du taux d'apprentissage. Observez et commentez. 

<div class="alert alert-block alert-info" style="margin-top : 0px">
    
__Solution 1.__
La présence d'aléa dans le choix de la fonction $f_{i_k}$ minimisée à l'étape $k$ introduit une compétition entre d'une part le gain moyen dû au fait que la moyenne des $-\nabla f_i(A)$ est $-\nabla F(A)$ qui est bien une direction de descente pour $F$ en $A$ et d'autre part l'effet aléatoire dû à la différence entre $\nabla F(A)$ et $\nabla f_i(A)$ qu'on peut quantifier par l'écart type
$$
E(A)=\left(\dfrac1{n_d}\sum_{i=0}^{n_d-1}\left\|\nabla f_i(A)-\nabla F(A)\right\|^2\right)^{1/2}.
$$
Cette différence a d'autant plus d'effet qu'on est proche d'un minimiseur local $A^*$. Dans ce cas $\nabla F(A^k)$ est petit et l'algorithme produit une marche aléatoire des $A^k$ au voisinage de $A^*$, avec en moyenne $\|A^k-A^*\|$ de l'ordre de $\alpha E(A^*)$.

On observe trois phases distinctes dans l'évolution de l'erreur.<br>
&nbsp;&nbsp;&nbsp;&nbsp;
(a) Une première phase où l'erreur totale décroît très rapidement (jusqu'autour de la vingt-cinquième itération ici). Dans cette phase le gain moyen domine clairement le bruit.<br>  
&nbsp;&nbsp;&nbsp;&nbsp;
(b) Une seconde phase où l'erreur décroît toujours en moyenne mais plus lentement et avec des oscillations de l'erreur dont l'amplitude est plus importante que le gain qu'il serait encore possible de réaliser sur l'erreur.


Les passage d'une phase à l'autre n'est pas brutal et le choix du point de passage d'une phase à l'autre nécessairement arbitraire.
    </div>

<h2><a id='II'></a>
2. Réduction du bruit par décroissance du taux d'apprentissage</h2> 

<a href="#toc">top</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#0">0</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#I">1.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#II">2.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#III">3.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#bot">bot.</a>

Pour réduire progressivement le bruit, on fait décroître le taux d'apprentissage au cours des _itérations_. L'itération $k$ devient :<br>
&nbsp;&nbsp;&nbsp;&nbsp;
Calculer le taux d'apprentissage $\alpha_k$.<br>
&nbsp;&nbsp;&nbsp;&nbsp; 
Choisir aléatoirement (et uniformément) $i\in\{0,\dots,n_d-1\}$.<br>
&nbsp;&nbsp;&nbsp;&nbsp; 
Faire :
$$
A\ \longleftarrow\ A - \alpha_k\nabla f_i(A),
$$

Nous proposons pour $\alpha_k$ la formule
$$\tag{1}
\alpha_k := \dfrac{\lambda\alpha_0}{\lambda+k}=\dfrac{\alpha_0}{1+ k/\lambda},
$$
où les paramètres $\alpha_0$ et $\lambda$ sont strictement positifs

**Exercice 2.** Donnez des arguments pour le choix des paramètres $\alpha_0$ et $\lambda$ (du moins donnez une méthode pour déterminer l'ordre de grandeur de paramètres conduisant à un algorithme efficace). 

On pourra vérifier le bien fondé de ces choix en testant la méthode (Exercice 3).

<div class="alert alert-block alert-info" style="margin-top : 0px">

**Solution 2.** Dans la formule (1), $\alpha_0$ est la valeur du premier taux d'apprentissage utilisé. Il est donc raissonnable de prendre pour $\alpha_0$ une valeur qui était efficace pour les premières itérations de la méthode à pas fixe.  Soit, ici,
$$
\alpha_0=5\cdot 10^{-2}.
$$

Si $\lambda$ est un entier alors $\alpha_{\lambda}=\alpha_0/2$. C'est à dire que le paramètre $\lambda$ correspond au nombre d'itérations qu'il faut pour que le taux d'accroissement soit divisé par 2. Plus généralement, pour un entier $m\ge0$,
$$
\alpha_{m\lambda}=\dfrac{\alpha_0}{m+1}.
$$
Pour profiter pleinement de la phase (a) de convergence rapide de la méthode du gradient stochastique testée dans la partie **1**, il ne faut pas que $\lambda$ soit trop petit. En notant $n_{\text{a}}$ le nombre d'itérations de cette phase (a), on demandera $\lambda\ge n_{\text{a}}$.<br>
On ne veut pas non plus que $\lambda$ soit trop grand pour que l'effet de réduction du bruit atténue au plus vite le phénomène de "marche aléatoire" de la phase (c).<br>
Un choix raisonnable semble de prendre $\lambda$ du même ordre de grandeur que $n_{\text{a}}$, disons $\lambda\simeq 2n_{\text{a}}$. Pour le problème particulier que nous traitons, la fin de la phase (a) se situe autour de 25 époques et comme une époque contient 1000 itérations, on a:
$$
n_{\text{a}}\simeq 25\cdot1000=2.5\cdot10^4.
$$
Dans l'implémentation proposée plus bas nous prenons exactement $\lambda=5\cdot10^4$.
</div>

**Exercice 3.** Implémentez la méthode du gradient stochastique avec le taux d'apprentissage variable donné par la formule (1).

In [ ]:
## Solution 3.a
# Paramètres
alpha0=0.005
fact_dec_alpha=1/(50*data.n) # inverse de lambda
Nepoch=600
Ndata=data.n
nepochplot=50

nplot_per_row = 4 # nombre de figures par ligne

# Initialisations
A=nn.create_rand()
nepoch=0
k=0
Erreur =[nn.total_loss(A,data,pb=pb)]
plotpos=0

In [ ]:
## Solution 3.b
# Boucle d'optimisation pour la methode du GS avec décroissance de alpha
for i_ in range(Nepoch):
    nepoch+=1
    for j_ in range(Ndata):
        i = nprd.randint(Ndata)
        x, y = data.X[i], data.Y[i]
        dA = nn.descent(A, x, y, alpha=alpha0/(1+fact_dec_alpha*k), pb=pb)
        nn.add(A, dA, output=False)
        k +=1
    # calcul de l'erreur et représentations graphiques
    if not nepoch%nepochplot:
        error = nn.total_loss_and_prediction(A,data,pb=pb)
        Erreur.append(error)
        if not plotpos: plt.figure(figsize=(16,4))
        plotpos+=1
        plt.subplot(1,nplot_per_row,plotpos)
        data.show_class(pred=True)
        nn.show_pred(A)
        pb.show_border('k--')
        plt.title(f"ep: {nepoch}, Loss: {error:1.5e}.", fontsize=12)
        if plotpos==nplot_per_row :  plt.show(); plotpos=0
    else:
        error = nn.total_loss(A,data,pb=pb)
        Erreur.append(error)

In [ ]:
## Solution 3.c
## Représentations graphiques de l'évolution de l'erreur au cours des époques.

#nn.show(A)

print(f"Erreur initiale : {Erreur[0]:1.5e}")

plt.figure(figsize=(16,6))
plt.subplot(121)
debut = 1
plt.plot(np.linspace(debut, nepoch,nepoch-debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")

plt.subplot(122)
debut = nepoch//2
plt.plot(np.linspace(debut, nepoch,nepoch-debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")
plt.show()

plt.figure(figsize=(16,6))
plt.subplot(121)
debut = 3*(nepoch//4)
plt.plot(np.linspace(debut, nepoch,nepoch - debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")

plt.subplot(122)
debut = 7*(nepoch//8)
plt.plot(np.linspace(debut, nepoch,nepoch - debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")
plt.show()

**Exercice 4.** Commentez les résultats numériques. Observe-t-on les effets décrits en cours ?

<div class="alert alert-block alert-info" style="margin-top : 0px">

__Solution 4.__ On observe deux phénomènes en comparaison de la méthode du gradien stochastique à taux d'apprentissage constant.<br>
    &nbsp;&nbsp;&nbsp;&nbsp; $\bullet$ D'une part la décroissance de l'erreur est plus régulière et légèrement plus lente.<br>
    &nbsp;&nbsp;&nbsp;&nbsp; $\bullet$ D'autre part, l'erreur continue de décroître: la phase (c) a disparu.
    
Remarquons aussi que le choix de $\alpha_k$ pourraît être modulé en fonction des oscillations observées de l'erreur.
</div>

<h2><a id='III'></a>3. Méthodes des mini-lots (mini-batch method)</h2> 

<a href="#toc">top</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#0">0</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#I">1.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#II">2.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#III">3.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#bot">bot.</a>

La méthode des mini-lots est une méthode intermédiare entre la méthode du gradient (full batch) et la méthode du gradient stochastique. Au lieu de considérer une seule fonction $f_{i_k}$ à l'étape $k$, on tire un lot de $n_k$ fonctions $f_{i_{k,0}},\dots,f_{i_{k,n_k-1}}$ et on fait 
$$
A^{k+1}\ \longleftarrow\ A^k-\alpha_k\dfrac1{n_k}\sum_{j=0}^{n_k-1}\nabla f_{i_{k,j}}(A_k).
$$
Notez que le calcul des $\nabla f_{i_{k,j}}(A_k)$ peut être effectué en parallèle.

**Exercice 5.** 

(i) Implémentez et testez la méthode de mini-batch sur le problème défini plus haut. On prendra $n_k=n_{\text{batch}}=30$ (indépendant de $k$) et dans un premier temps un taux d'apprentissage constant. 

On rappelle que le nombre d'itérations par époque est $\lfloor n_d/n_{\text{batch}}\rfloor$. 

(ii) Comparez la méthode de mini-batch et la méthode de gradient stochastique pur. Est-ce que les taux d'apprentissage permettant la convergence sont les mêmes que dans le cas du gradient stochastique pur ? Pour un même taux d'apprentissage, y-a-il une différence dans l'évolution de l'erreur au cour des itérations?

(iii) Implémentez la méthode du mini-batch avec $\alpha_k$ donné par la formule (1).

(iv) Quels sonts les conclusions générales que vous tirez du TP ? 

In [ ]:
## Solution 5.i.a
# Paramètres
alpha0=0.01
Nepoch=600
Nbatch=30
Ndata=data.n
nepochplot=50
Niter_per_epoch=Ndata//Nbatch
fact_dec_alpha=0 
#fact_dec_alpha=1/(600*Niter_per_epoch)

nplot_per_row = 4 # nombre de figures par ligne

# Initialisations
A=nn.create_rand()
nepoch=0
k=0
Erreur =[nn.total_loss(A,data,pb=pb)]
plotpos=0

In [ ]:
## Solution 5.i.b
# Boucle d'optimisation pour la methode du GS avec décroissance de alpha
for i_ in range(Nepoch):
    nepoch+=1
    for j_ in range(Niter_per_epoch):
        I = nprd.choice(Ndata, Nbatch)
        DA=nn.create_zero()
        for i in I:
            x, y = data.X[i], data.Y[i]
            nn.descent(A, x, y, alpha=1, B=DA, pb=pb)
        nn.add(A, DA, c=alpha0/(Nbatch*(1+fact_dec_alpha*k)), output=False)
        k +=1
    # calcul de l'erreur et représentations graphiques
    if not nepoch%nepochplot:
        error = nn.total_loss_and_prediction(A,data,pb=pb)
        Erreur.append(error)
        if not plotpos: plt.figure(figsize=(16,4))
        plotpos+=1
        plt.subplot(1,nplot_per_row,plotpos)
        data.show_class(pred=True)
        nn.show_pred(A)
        pb.show_border('k--')
        plt.title(f"ep: {nepoch}, Loss: {error:1.5e}.", fontsize=12)
        if plotpos==nplot_per_row :  plt.show(); plotpos=0
    else:
        error = nn.total_loss(A,data,pb=pb)
        Erreur.append(error)

In [ ]:
## Solution 5.i.c
## Représentations graphiques de l'évolution de l'erreur au cours des époques.

#nn.show(A)

print(f"Erreur initiale : {Erreur[0]:1.5e}")

plt.figure(figsize=(16,6))
plt.subplot(121)
debut = 1
plt.plot(np.linspace(debut, nepoch,nepoch-debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")

plt.subplot(122)
debut = nepoch//2
plt.plot(np.linspace(debut, nepoch,nepoch-debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")
plt.show()

plt.figure(figsize=(16,6))
plt.subplot(121)
debut = 3*(nepoch//4)
plt.plot(np.linspace(debut, nepoch,nepoch - debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")

plt.subplot(122)
debut = 7*(nepoch//8)
plt.plot(np.linspace(debut, nepoch,nepoch - debut + 1),Erreur[debut:])
plt.title("Erreur en fonction du nombre d'époques")
plt.show()

<div class="alert alert-block alert-info" style="margin-top : 0px">
    
**Solution 5.ii.** 

On remarque que la méthode de mini-lots reste stable avec un taux d'apprentissage plus important que pour la méthode de gradient stochastique pure. Cela est dû au fait que la moyenne
$$
-\dfrac1{n_k}\sum_{j=0}^{n_k-1}\nabla f_{i_{k,j}}(A^k)
$$
est plus souvent une direction de descente pour $F(A^k)$ que le seul $-\nabla f_{i_k}(A^k)$. 

Pour les mêmes raisons les oscillations de l'erreur sont moins importantes dans cas de la méthode de mini-lots.

Cela présente un inconvénient. En effet les oscillations aléatoires permettent d'explorer le paysage énergétique. On pourra remarquer que dans le cas de la méthode des mini-lots, à la fin de la phase de décroissance rapide, l'erreur est plus élevée que pour la méthode de gradientstochastique pure.
     </div>

<div class="alert alert-block alert-info" style="margin-top : 0px">
    
**Solution 5.iii.** Il suffit de changer la définition de _fact_dec_alpha_ dans la partie _\# Paramètres_ en commentant la ligne courante et décommantant la suivante.
    </div>

<div class="alert alert-block alert-info" style="margin-top : 0px">
    
**Solution 5.iv.** 

Nous avons vu deux méthodes de réduction du bruit : la méthode des taux d'appentissage décroissants et la méthode des mini-lots. Ces deux méthodes peuvent être avantageusement combinées. 

Dans le cas de la méthode des mini-lots, le calcul de la direction de descente peut être parallélisé.

Les méthodes de réduction de bruit limitent l'exploration du paysage énergétique. Il ne faut donc pas que la réduction soit trop forte ou intervienne trop tôt.


Il faut préter attention au choix du taux d'apprentissage initial $\alpha_0$. Avec un taux trop grand, l'algorithme est instable. Avec un taux trop petit la convergence est très lente. L'intervalle de taux initiaux efficaces dépend du problème, du type de réseau de neurones considérés et de la taille des mini-lots.
    </div>

<a href="#toc">top</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#0">0</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#I">1.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#II">2.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#III">3.</a>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="#bot">bot.</a>
<a id='bot'></a>